In [1]:
import socket
import pickle
import random
import tkinter as tk
from tkinter import ttk, messagebox
from ttkthemes import ThemedStyle
from PIL import Image, ImageTk

def CreatePolynomial(coefficients, x, prime_modulus):
    polynomial = 0
    for i, coef in enumerate(coefficients):
        polynomial = (polynomial + coef * (x ** i)) % prime_modulus
    return polynomial

def CreateShares(secret, threshold, prime_modulus, NumberOfShares):
    coefficients = [secret] + [random.randint(1, prime_modulus - 1) for _ in range(threshold - 1)]

    shares = []
    for i in range(1, NumberOfShares + 1):
        x = random.randint(1, prime_modulus - 1)
        y = CreatePolynomial(coefficients, x, prime_modulus) % prime_modulus
        shares.append((x, y))

    return shares

def lagrange_interpolation(shares, prime_modulus):
    secret_reconstructed = 0
    for i, yi in shares:
        numerator = denominator = 1
        for j, yj in shares:
            if i != j:
                numerator = (numerator * -j) % prime_modulus
                denominator = (denominator * (i - j)) % prime_modulus
        lagrange_term = (yi * numerator * pow(denominator, -1, prime_modulus)) % prime_modulus
        secret_reconstructed = (secret_reconstructed + lagrange_term) % prime_modulus

    return secret_reconstructed

def handle_client(client_socket):
    data = client_socket.recv(1024)
    client_request = pickle.loads(data)

    # Shamir's Secret Sharing
    secret, threshold, prime_modulus = client_request
    shares = CreateShares(secret, threshold, prime_modulus, 5)
    reconstructed_secret = lagrange_interpolation(shares[:threshold], prime_modulus)

    # Send the result back to the client
    result_data = pickle.dumps(reconstructed_secret)
    client_socket.sendall(result_data)
    client_socket.close()

def run_server():
    HOST = '127.0.0.1'
    PORT = 8080

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((HOST, PORT))
        server_socket.listen()

        print(f"Server listening on {HOST}:{PORT}")

        while True:
            conn, addr = server_socket.accept()
            with conn:
                print(f"Connected by {addr}")
                handle_client(conn)

if __name__ == "__main__":
    run_server()


Server listening on 127.0.0.1:8080
